In [1]:
from sdssdb.peewee.sdss5db import targetdb

# connect to targetdb
targetdb.database.connect_from_parameters(user='sdss_user', password='4-replicant',
                                          host='localhost',
                                          port=7500)

True

In [37]:
from peewee import *
import numpy as np

##MEG & JON: 3 changes to be made
# set search radius based on field size for APO or LCO
observatory = 'APO'
if observatory == 'APO':
    r_search = 1.49
else:
    r_search = 0.95
skies = 50
# specify the field center and position angle
#MEG & JON: CHANGE FOR FIELD
#from: https://wiki.sdss.org/display/BHM/Reverberation+Mapping
##XMM LSS #s
racen = 35.708333
deccen = -5.050000
fieldname = "XMMLSS"
##SDSS-RM
# racen = 213.7041666
# deccen = 53.083333
# fieldname = "RM"
# ##COSMOS
# racen = 10.0*15
# deccen = 2.20
# fieldname = "COSMOS"

In [38]:
cartons = ['bhm_rm_known_spec', 'bhm_rm_core','bhm_rm_var', 'bhm_rm_ancillary','ops_std_eboss']#'ops_sky_apogee',
plan = '0.5.0'
# get all of the targets in the commissioning carton near the field center
all_targs = (targetdb.Target.select(targetdb.Target.catalogid,
                                   targetdb.Target.ra,
                                   targetdb.Target.dec,
                                   targetdb.Target.pk,
                                   targetdb.CartonToTarget.priority,
                                   targetdb.CartonToTarget.value,
                                   targetdb.Cadence.label,
                                   targetdb.Carton.carton,
                                   targetdb.CartonToTarget.pk,
                                   targetdb.Magnitude.g,
                                   targetdb.Magnitude.r,
                                   targetdb.Magnitude.i,
                                   targetdb.Magnitude.bp,
                                   targetdb.Magnitude.gaia_g,
                                   targetdb.Magnitude.rp,
                                   targetdb.Magnitude.h,
                                   targetdb.Category.label,
                                   targetdb.Carton.program,
                                   targetdb.Instrument.label)
                    .join(targetdb.CartonToTarget)
                    .join(targetdb.Cadence, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Carton)
                    .join(targetdb.Category)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Magnitude, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Instrument)
                    .where((targetdb.Carton.carton.in_(cartons)) &
                           (targetdb.Target.cone_search(racen, deccen, r_search))
                           ))

ops_carton = ['ops_sky_boss']
ops_plan = '0.1.0'
ops_query = (targetdb.Target.select(targetdb.Target.catalogid,
                                   targetdb.Target.ra,
                                   targetdb.Target.dec,
                                   targetdb.Target.pk,
                                   targetdb.CartonToTarget.priority,
                                   targetdb.CartonToTarget.value,
                                   targetdb.Cadence.label,
                                   targetdb.Carton.carton,
                                   targetdb.CartonToTarget.pk,
                                   targetdb.Magnitude.g,
                                   targetdb.Magnitude.r,
                                   targetdb.Magnitude.i,
                                   targetdb.Magnitude.bp,
                                   targetdb.Magnitude.gaia_g,
                                   targetdb.Magnitude.rp,
                                   targetdb.Magnitude.h,
                                   targetdb.Category.label,
                                   targetdb.Carton.program,
                                   targetdb.Instrument.label)
                    .join(targetdb.CartonToTarget)
                    .join(targetdb.Cadence, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Carton)
                    .join(targetdb.Category)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Magnitude, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Instrument)
                    .where((targetdb.Carton.carton.in_(ops_carton)) &
                           (targetdb.Target.cone_search(racen, deccen, r_search))
                           ))

# grab the results
catalogid, ra, dec, target_pk, priority, value, cadences, carton, carton_to_target_pk, g, r, i, bp, gaia_g, rp, h, category, program, instrument  = map(list, zip(*list(all_targs.tuples())))
catalogid = np.array(catalogid, dtype=np.int64)
ra = np.array(ra)
dec = np.array(dec)
target_pk = np.array(target_pk, dtype=np.int64)
priority = np.array(priority)
value = np.array(value)
carton = np.array(carton)
carton_to_target_pk = np.array(carton_to_target_pk)
magnitudes = np.zeros((len(g),7))
magnitudes[:, 0] = g
magnitudes[:, 1] = r
magnitudes[:, 2] = i
magnitudes[:, 3] = bp
magnitudes[:, 4] = gaia_g
magnitudes[:, 5] = rp
magnitudes[:, 6] = h
category = np.array(category)
program = np.array(program)
instrument = np.array(instrument)

# grab the results, but for the sky stuff
catalogid2, ra2, dec2, target_pk2, priority2, value2, cadences2, carton2, carton_to_target_pk2, g2, r2, i2, bp2, gaia_g2, rp2, h2, category2, program2, instrument2 = map(list, zip(*list(ops_query.tuples())))
catalogid2 = np.array(catalogid2, dtype=np.int64)
ra2 = np.array(ra2)
dec2 = np.array(dec2)
target_pk2 = np.array(target_pk2, dtype=np.int64)
priority2 = np.array(priority2)
value2 = np.array(value2)
carton2 = np.array(carton2)
carton_to_target_pk2 = np.array(carton_to_target_pk2)
magnitudes2 = np.zeros((len(g2),7))
magnitudes2[:, 0] = g2
magnitudes2[:, 1] = r2
magnitudes2[:, 2] = i2
magnitudes2[:, 3] = bp2
magnitudes2[:, 4] = gaia_g2
magnitudes2[:, 5] = rp2
magnitudes2[:, 6] = h2
category2 = np.array(category2)
program2 = np.array(program2)
instrument2 = np.array(instrument2)


#append together
catalogid = np.append(catalogid, catalogid2)
ra = np.append(ra, ra2)
dec = np.append(dec, dec2)
target_pk = np.append(target_pk, target_pk2)
priority = np.append(priority, priority2)
value = np.append(value, value2)
carton = np.append(carton, carton2)
carton_to_target_pk = np.append(carton_to_target_pk, carton_to_target_pk2)
g = np.append(g, g2)
r = np.append(r, r2)
i = np.append(i, i2)
bp = np.append(bp, bp2)
gaia_g = np.append(gaia_g, gaia_g2)
rp = np.append(rp, rp2)
h = np.append(h, h2)

magnitudes = np.append(magnitudes, magnitudes2)
program = np.append(program, program2)
category = np.append(category, category2)
instrument = np.append(instrument, instrument2)
tag = np.full(len(catalogid), '0.3.5')
plan = np.full(len(catalogid), '0.5.4')

# specify the instrument for each target
# instrument = np.zeros(len(carton), dtype='<U10')

# #MEG & JON: CHANGE FOR FIELD
# instrument[carton == 'ops_sky_boss'] = 'BOSS'
# instrument[carton == 'ops_std_eboss'] = 'BOSS'
# instrument[carton == 'bhm_rm_known_spec'] = 'BOSS'
# instrument[carton == 'bhm_rm_core'] = 'BOSS'
# instrument[carton == 'bhm_rm_var'] = 'BOSS'
# instrument[carton == 'bhm_rm_ancillary'] = 'BOSS'

In [39]:
print(type(category[0]))

<class 'numpy.str_'>


In [40]:
from astropy.table import Table, Column
tab = Table()
tab.add_column(Column(name="catalogid", data=catalogid, dtype='int64'))
tab.add_column(Column(name="ra", data=ra, dtype='float32'))
tab.add_column(Column(name="dec", data=dec, dtype='float32'))
tab.add_column(Column(name="target_pk", data=target_pk, dtype='int64'))
tab.add_column(Column(name="priority", data=priority, dtype='float32'))
tab.add_column(Column(name="value", data=value, dtype='float32'))
tab.add_column(Column(name="carton", data=carton))
tab.add_column(Column(name="instrument", data=instrument, dtype='<U10'))
tab.add_column(Column(name="carton_to_target_pk", data=carton_to_target_pk, dtype='int64'))
tab.add_column(Column(name="g", data=g, dtype='float32'))
tab.add_column(Column(name="r", data=r, dtype='float32'))
tab.add_column(Column(name="i", data=i, dtype='float32'))
tab.add_column(Column(name="bp", data=bp, dtype='float32'))
tab.add_column(Column(name="gaia_g", data=gaia_g, dtype='float32'))
tab.add_column(Column(name="rp", data=rp, dtype='float32'))
tab.add_column(Column(name="h", data=h, dtype='float32'))
tab.add_column(Column(name="category", data=category))
tab.add_column(Column(name="tag", data=tag, dtype='<U10'))
tab.add_column(Column(name="plan", data=plan, dtype='<U10'))
tab.add_column(Column(name="program", data=program))
tab.write("BHM_cartons_{}_survey_v0.5.fits".format(fieldname), format='fits', overwrite=True)

In [41]:
data = Table.read("./BHM_cartons_{}_survey_v0.5.fits".format(fieldname), format='fits')
data

catalogid,ra,dec,target_pk,priority,value,carton,instrument,carton_to_target_pk,g,r,i,bp,gaia_g,rp,h,category,tag,plan,program
int64,float32,float32,int64,float32,float32,bytes17,bytes10,int64,float32,float32,float32,float32,float32,float32,float32,bytes13,bytes10,bytes10,bytes7
27021597934348191,35.066505,-6.148681,246495109,5300.0,nan,ops_std_eboss,BOSS,462294627,16.484657,16.262238,16.135529,16.449074,16.222317,15.823082,15.021,standard_boss,0.3.5,0.5.4,ops_std
27021597934353365,36.391823,-5.9646945,246495180,5300.0,nan,ops_std_eboss,BOSS,462295618,16.88191,16.625328,16.549017,16.848253,16.623844,16.234093,15.579,standard_boss,0.3.5,0.5.4,ops_std
27021597934353857,36.73269,-5.76863,246495188,5300.0,nan,ops_std_eboss,BOSS,462295620,16.748152,16.435757,16.332052,16.70827,16.437359,16.004848,15.241,standard_boss,0.3.5,0.5.4,ops_std
27021597934475735,34.675846,-4.939207,246496937,5300.0,nan,ops_std_eboss,BOSS,462292772,17.113632,16.831245,16.79603,17.083937,16.83837,16.41806,15.663,standard_boss,0.3.5,0.5.4,ops_std
27021597934476383,34.80909,-4.493158,246496945,5300.0,nan,ops_std_eboss,BOSS,462292872,16.641798,16.423796,16.310085,16.622835,16.375942,15.988426,15.29,standard_boss,0.3.5,0.5.4,ops_std
27021597934477731,35.513508,-4.5761,246496963,5300.0,nan,ops_std_eboss,BOSS,462293873,16.417242,16.105057,15.997835,16.373522,16.111666,15.68103,14.844,standard_boss,0.3.5,0.5.4,ops_std
27021597934479325,36.482265,-5.553629,246496990,5300.0,nan,ops_std_eboss,BOSS,462295706,16.441372,16.057964,15.917111,16.37014,16.071367,15.587784,14.733,standard_boss,0.3.5,0.5.4,ops_std
27021597934480917,35.747036,-4.5849276,246497004,5300.0,nan,ops_std_eboss,BOSS,462293874,16.856415,16.489721,16.335539,16.804138,16.49805,16.027735,15.328,standard_boss,0.3.5,0.5.4,ops_std
27021597934481174,36.09843,-4.6852045,246497008,5300.0,nan,ops_std_eboss,BOSS,462293879,17.097435,16.872766,16.716433,17.076422,16.81653,16.41665,15.79,standard_boss,0.3.5,0.5.4,ops_std


In [31]:
np.unique(data['value'])

1000.0
nan


In [43]:
np.unicode("0.5.3")

[WARNING]: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here. (DeprecationWarning)


'0.5.3'